<a href="https://colab.research.google.com/github/yilin2718/search-product-reviews-weaviate/blob/main/cap_proj_NER_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spacy based entity extraction and ambiguation.

100K products and 100K vendors used in the MVP. 

Based on:

Concise Concepts:   
https://github.com/Pandora-Intelligence/concise-concepts

POS tagging:
https://spacy.io/usage/linguistic-features

Spacy Span Categorizer (detecting entities within entities)
https://spacy.io/api/spancategorizer




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/FBproject/


/content/drive/MyDrive/FBproject


In [ ]:
pwd

'/content/drive/MyDrive/FBproject'

In [ ]:
pip install Spacy==3.2

     |████████████████████████████████| 6.0 MB 28.1 MB/s 
     |████████████████████████████████| 653 kB 52.9 MB/s 
     |████████████████████████████████| 10.1 MB 51.6 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 181 kB 65.0 MB/s 
     |████████████████████████████████| 457 kB 54.8 MB/s 
     |████████████████████████████████| 58 kB 7.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    

**NER customization**


In [ ]:
#helper functions
# we have some bad data - name is float so need to check for it

def convert_to_lowerCase (df_in, col_name):
  for i in range(len(df_in)):
    if isinstance (df_in.iloc[i][col_name], float):
      print (i, df_in.iloc[i][col_name])
      c
      df_in.iloc[i][col_name] = str( df_in.iloc[i][col_name] )
    df_in.iloc[i][col_name] = str.lower(df_in.iloc[i][col_name])
  return df_in

all_prods_df = convert_to_lowerCase (all_prods_df, 'productName')
all_vendors_df = convert_to_lowerCase (all_vendors_df, 'name')
all_cats_df = convert_to_lowerCase (all_cats_df, 'name')
#all_prods_df['productName'] = all_prods_df['productName'].apply(str.lower)
#all_cats_df ['name'] = all_cats_df['name'].apply(str.lower)
#all_vendors_df ['name'] = all_vendors_df['name'].apply(str.lower)

In [ ]:
# remove company, ltd etc from vendor names

frag_list = ['company', 'ltd', 'inc', 'inc.', 'labs', 'gmbh']
frag_filter_list = ['and', 'in', 'management', 'software', 'i', 'to']

frag_vendorNames_list =[]
for v_name in all_vendors_list:
  # check for .inc or .Inc
  if any( ele in v_name for ele in frag_list ):
    fname = v_name.split(" ")[0].rstrip(',')
    if fname not in frag_filter_list:
      frag_vendorNames_list.append(fname)
      

In [ ]:
pattern_prod_list = []
for iprod in all_prods_list:
  if isinstance(iprod, float):
    continue
  prd_list = iprod.split(" ")
  if (len(prd_list) == 1):
    pattern_prod_list.append( [{"TEXT" : iprod}])
  else:
    tmp_list = []
    for iwrd in prd_list:
      tmp_list.append ({"TEXT" : iwrd})
    pattern_prod_list.append(tmp_list)

###
prod_patterns = []
for x in pattern_prod_list:
  prod_patterns.append( {"label" : "PRODUCT", "pattern" : x })

pattern_vendor_list = []
for iprod in all_vendors_list:
  if isinstance(iprod, float):
    continue
  prd_list = iprod.split(" ")
  if (len(prd_list) == 1):
    pattern_vendor_list.append( [{"TEXT" : iprod}])
  else:
    tmp_list = []
    for iwrd in prd_list:
      tmp_list.append ({"TEXT" : iwrd})
    pattern_vendor_list.append(tmp_list)

###
vendor_patterns = []
for x in pattern_vendor_list:
  vendor_patterns.append( {"label" : "VENDOR", "pattern" : x })

***Develop A Custom Entity Ruler Model***

In [ ]:
from spacy.lang.en import English

nlp = English()
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(prod_patterns)
ruler.add_patterns(vendFrag_patterns) 
ruler.add_patterns(vendor_patterns)
ruler.add_patterns(cat_patterns)


doc1 = nlp(str.lower("I want compare Apache Hama PageManager AppointMate should we buy them from Assembrix who sells Sports League Software "))
print([(ent.text, ent.label_) for ent in doc1.ents])

# save customzied model to disk
ruler.to_disk("./run1_all_patterns_may5.jsonl")

**Read Saved Model from Disk**



In [ ]:
from spacy.lang.en import English
from pathlib import Path

fpath = Path("/content/drive/MyDrive/FBproject/run1_all_patterns_may5.jsonl")

nlp2 = English()
ruler2 = nlp2.add_pipe("entity_ruler").from_disk(fpath)

doc2 = nlp2(str.lower("I want to buy PHPjabbers Equipment Rental Software from Microsoft"))
print([(ent.text, ent.label_) for ent in doc2.ents])

[('phpjabbers equipment rental software', 'VENDOR_FRAGMENT'), ('microsoft', 'VENDOR')]


In [ ]:
#Text = 'Cheaper options than Salesforce' 
#Text = 'Is Flaticon the best product in Bug Tracking Software' 
#Text = "Should we buy Web4Realty or some other product in HVAC Software "
#Text = 'compare Automilez and Valtari'
Text = 'I want compare PageManager AppointMate should we buy them from Assembrix who sells Sports League Software '
#Text = 'is Slack or Zoom or Google better?'

doc3 = nlp2(str.lower(Text))
print([(ent.text, ent.label_) for ent in doc3.ents])

[('pagemanager', 'VENDOR_FRAGMENT'), ('appointmate', 'VENDOR_FRAGMENT'), ('we', 'VENDOR'), ('them', 'VENDOR'), ('sports league software', 'CATEGORY')]


In [ ]:
## production quality code

from spacy.lang.en import English
class QueryParse ():

  def __init__(self):
    self.nlp_model = English()
    ruler = self.nlp_model.add_pipe("entity_ruler").from_disk("./run1_all_patterns_may5.jsonl")

    
  def parse_query (self, qry_request, context):
    """
    parse user query
    """
    doc = self.nlp_model(str.lower(qry_request))
    phrase_list = []
    label_list = []
    comb_list = []
    for ent in doc.ents:
      phrase_list.append(ent.text)
      label_list.append(ent.label_)
      comb_list.append([ent.label_, ent.text])
    
    if len(label_list) == 1:
      return comb_list[0]
    
    return comb_list


In [ ]:
lab_service = QueryParse ( )
xx = lab_service.parse_query ("phpjabbers knowledge base builder", 'xx')
print (xx)

['PRODUCT', 'phpjabbers knowledge base builder']
